In [1]:
from datasets import load_dataset, load_metric
from transformers import (AutoTokenizer, GPT2Tokenizer, AutoModelForSequenceClassification, 
                          TrainingArguments, Trainer, AutoModelForCausalLM, GPT2LMHeadModel, AutoModelWithLMHead,
                         DataCollatorForLanguageModeling, pipeline)
import torch
import numpy as np
import os
import tensorflow as tf

## Preparing data (Emotion dataset by Twitter)

In [2]:
emotion_dataset = load_dataset("emotion")

Using custom data configuration default
Reusing dataset emotion (/home/avsorokina/.cache/huggingface/datasets/emotion/default/0.0.0/348f63ca8e27b3713b6c04d723efe6d824a56fb3d1449794716c0f0296072705)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
# prepare individual datasets for every emotion

emotion_dataset_sadness = emotion_dataset.filter(lambda record: record['label']==0)
emotion_dataset_joy = emotion_dataset.filter(lambda record: record['label']==1)
emotion_dataset_love = emotion_dataset.filter(lambda record: record['label']==2)
emotion_dataset_anger = emotion_dataset.filter(lambda record: record['label']==3)
emotion_dataset_fear = emotion_dataset.filter(lambda record: record['label']==4)
emotion_dataset_surprise = emotion_dataset.filter(lambda record: record['label']==5)

Loading cached processed dataset at /home/avsorokina/.cache/huggingface/datasets/emotion/default/0.0.0/348f63ca8e27b3713b6c04d723efe6d824a56fb3d1449794716c0f0296072705/cache-3315adf707f1f657.arrow
Loading cached processed dataset at /home/avsorokina/.cache/huggingface/datasets/emotion/default/0.0.0/348f63ca8e27b3713b6c04d723efe6d824a56fb3d1449794716c0f0296072705/cache-4045f2553a29b14d.arrow
Loading cached processed dataset at /home/avsorokina/.cache/huggingface/datasets/emotion/default/0.0.0/348f63ca8e27b3713b6c04d723efe6d824a56fb3d1449794716c0f0296072705/cache-a4014524468efda4.arrow
Loading cached processed dataset at /home/avsorokina/.cache/huggingface/datasets/emotion/default/0.0.0/348f63ca8e27b3713b6c04d723efe6d824a56fb3d1449794716c0f0296072705/cache-67a76582b0169275.arrow
Loading cached processed dataset at /home/avsorokina/.cache/huggingface/datasets/emotion/default/0.0.0/348f63ca8e27b3713b6c04d723efe6d824a56fb3d1449794716c0f0296072705/cache-7d3e782fefa500ca.arrow
Loading cached 

In [4]:
# define tolenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [5]:
tokenizer.pad_token = tokenizer.eos_token

In [6]:
def tokenize_func(record):
    return tokenizer(record["text"], padding = "max_length", truncation = True, max_length = 128)

In [7]:
sadness_tokenized = emotion_dataset_sadness.map(tokenize_func)
joy_tokenized = emotion_dataset_joy.map(tokenize_func)
love_tokenized = emotion_dataset_love.map(tokenize_func)
anger_tokenized = emotion_dataset_anger.map(tokenize_func)
fear_tokenized = emotion_dataset_fear.map(tokenize_func)
surprise_tokenized = emotion_dataset_surprise.map(tokenize_func)

Loading cached processed dataset at /home/avsorokina/.cache/huggingface/datasets/emotion/default/0.0.0/348f63ca8e27b3713b6c04d723efe6d824a56fb3d1449794716c0f0296072705/cache-2cb30565284d9a62.arrow
Loading cached processed dataset at /home/avsorokina/.cache/huggingface/datasets/emotion/default/0.0.0/348f63ca8e27b3713b6c04d723efe6d824a56fb3d1449794716c0f0296072705/cache-15b4bd57b7dcf299.arrow


  0%|          | 0/581 [00:00<?, ?ex/s]

Loading cached processed dataset at /home/avsorokina/.cache/huggingface/datasets/emotion/default/0.0.0/348f63ca8e27b3713b6c04d723efe6d824a56fb3d1449794716c0f0296072705/cache-76f09b1eb73890f1.arrow
Loading cached processed dataset at /home/avsorokina/.cache/huggingface/datasets/emotion/default/0.0.0/348f63ca8e27b3713b6c04d723efe6d824a56fb3d1449794716c0f0296072705/cache-67881207f4809dae.arrow
Loading cached processed dataset at /home/avsorokina/.cache/huggingface/datasets/emotion/default/0.0.0/348f63ca8e27b3713b6c04d723efe6d824a56fb3d1449794716c0f0296072705/cache-867718b4290685d2.arrow
Loading cached processed dataset at /home/avsorokina/.cache/huggingface/datasets/emotion/default/0.0.0/348f63ca8e27b3713b6c04d723efe6d824a56fb3d1449794716c0f0296072705/cache-92070843494168e5.arrow
Loading cached processed dataset at /home/avsorokina/.cache/huggingface/datasets/emotion/default/0.0.0/348f63ca8e27b3713b6c04d723efe6d824a56fb3d1449794716c0f0296072705/cache-9cacb76c255a0ddf.arrow
Loading cached 

In [8]:
# remove useless columns + rename + change format

sadness_tokenized = sadness_tokenized.remove_columns(['text']).rename_column('label', 'labels').with_format('torch')
joy_tokenized = joy_tokenized.remove_columns(['text']).rename_column('label', 'labels').with_format('torch')
love_tokenized = love_tokenized.remove_columns(['text']).rename_column('label', 'labels').with_format('torch')
anger_tokenized = anger_tokenized.remove_columns(['text']).rename_column('label', 'labels').with_format('torch')
fear_tokenized = fear_tokenized.remove_columns(['text']).rename_column('label', 'labels').with_format('torch')
surprise_tokenized = surprise_tokenized.remove_columns(['text']).rename_column('label', 'labels').with_format('torch')


In [9]:
sadness_train_dataset, sadness_eval_dataset = sadness_tokenized["train"], sadness_tokenized["test"]
joy_train_dataset, joy_eval_dataset = joy_tokenized["train"], joy_tokenized["test"]
love_train_dataset, love_eval_dataset = love_tokenized["train"], love_tokenized["test"]
anger_train_dataset, anger_eval_dataset = anger_tokenized["train"], anger_tokenized["test"]
fear_train_dataset, fear_eval_dataset = fear_tokenized["train"], fear_tokenized["test"]
surprise_train_dataset, surprise_eval_dataset = surprise_tokenized["train"], surprise_tokenized["test"]

# Modelling 

In [10]:
training_args = TrainingArguments(
                    output_dir="./emotions",
                    overwrite_output_dir=True,
#                     do_train=True,
#                     prediction_loss_only=True,
#                     evaluation_strategy = 'epoch',
                    num_train_epochs=10, 
                    per_device_train_batch_size=2,
                    per_device_eval_batch_size=4, 
                    logging_steps = 500,
                    save_steps=500,
                    warmup_steps=500,
#                     save_strategy = 'epoch',
#                     learning_rate = 0.05,
#                     max_steps = -1,
#                     weight_decay=0.01,
#                     load_best_model_at_end=True,
#                     resume_from_checkpoint = True
#                     report_to="wandb",  # enable logging to W&B
                    )

## Sadness

In [67]:
model_sadness = AutoModelWithLMHead.from_pretrained("gpt2")
model_sadness.config.pad_token_id = model_sadness.config.eos_token_id
tokenizer_sadness = AutoTokenizer.from_pretrained('gpt2') 
tokenizer_sadness.pad_token = tokenizer_sadness.eos_token

data_collator_sadness = DataCollatorForLanguageModeling(
        tokenizer=tokenizer_sadness, mlm=False,
    )
trainer_sadness = Trainer(
    model=model_sadness, args=training_args, 
    data_collator=data_collator_sadness,
    train_dataset=sadness_train_dataset)
trainer_sadness.train()

trainer_sadness.save_model("emotion_fine_tuned/gpt2_sadness")
tokenizer_sadness.save_pretrained("emotion_fine_tuned/gpt2_sadness")

loading configuration file https://huggingface.co/gpt2/resolve/main/config.json from cache at /home/avsorokina/.cache/huggingface/transformers/fc674cd6907b4c9e933cb42d67662436b89fa9540a1f40d7c919d0109289ad01.7d2e0efa5ca20cef4fb199382111e9d3ad96fd77b849e1d4bed13a66e1336f51
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_

In [11]:
tokenizer_sadness = AutoTokenizer.from_pretrained("emotion_fine_tuned/gpt2_sadness")
sadness_model = AutoModelWithLMHead.from_pretrained("emotion_fine_tuned/gpt2_sadness")

/home/avsorokina/anaconda3/lib/python3.8/site-packages/transformers/models/auto/modeling_auto.py:742: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [12]:
gpt2_finetune_sadness = pipeline('text-generation', model=sadness_model, tokenizer=tokenizer_sadness)

## Joy

In [49]:
tokenizer_joy = AutoTokenizer.from_pretrained('gpt2') 
model_joy = AutoModelWithLMHead.from_pretrained("gpt2")
model_joy.config.pad_token_id = model_joy.config.eos_token_id
tokenizer_joy.pad_token = tokenizer_joy.eos_token

data_collator_joy = DataCollatorForLanguageModeling(
        tokenizer=tokenizer_joy, mlm=False,
    )
trainer_joy = Trainer(
    model=model_joy, 
    args=training_args, 
    data_collator=data_collator_joy,
    train_dataset=joy_train_dataset)
trainer_joy.train()

trainer_joy.save_model("emotion_fine_tuned/gpt2_joy")
tokenizer_joy.save_pretrained("emotion_fine_tuned/gpt2_joy")

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/gpt2/resolve/main/config.json from cache at /home/avsorokina/.cache/huggingface/transformers/fc674cd6907b4c9e933cb42d67662436b89fa9540a1f40d7c919d0109289ad01.7d2e0efa5ca20cef4fb199382111e9d3ad96fd77b849e1d4bed13a66e1336f51
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_t

Step,Training Loss
500,4.561300
1000,4.288000
1500,4.262100
2000,4.218600
2500,4.189300
3000,3.820100
3500,3.588700
4000,3.576200
4500,3.606700
5000,3.594900


Saving model checkpoint to ./emotions/checkpoint-500
Configuration saved in ./emotions/checkpoint-500/config.json
Model weights saved in ./emotions/checkpoint-500/pytorch_model.bin
Saving model checkpoint to ./emotions/checkpoint-1000
Configuration saved in ./emotions/checkpoint-1000/config.json
Model weights saved in ./emotions/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to ./emotions/checkpoint-1500
Configuration saved in ./emotions/checkpoint-1500/config.json
Model weights saved in ./emotions/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to ./emotions/checkpoint-2000
Configuration saved in ./emotions/checkpoint-2000/config.json
Model weights saved in ./emotions/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to ./emotions/checkpoint-2500
Configuration saved in ./emotions/checkpoint-2500/config.json
Model weights saved in ./emotions/checkpoint-2500/pytorch_model.bin
IOPub message rate exceeded.
The notebook server will temporarily stop sending outp

In [13]:
tokenizer_joy = AutoTokenizer.from_pretrained("emotion_fine_tuned/gpt2_joy")
joy_model = AutoModelWithLMHead.from_pretrained("emotion_fine_tuned/gpt2_joy")

In [14]:
gpt2_finetune_joy = pipeline('text-generation', model=joy_model, tokenizer=tokenizer_joy)

## Love

In [23]:
tokenizer_love = AutoTokenizer.from_pretrained('gpt2') 
model_love = AutoModelWithLMHead.from_pretrained("gpt2")
model_love.config.pad_token_id = model_love.config.eos_token_id
tokenizer_love.pad_token = tokenizer_love.eos_token

data_collator_love = DataCollatorForLanguageModeling(
        tokenizer=tokenizer_love, mlm=False,
    )
trainer_love = Trainer(
    model=model_love, 
    args=training_args, 
    data_collator=data_collator_love,
    train_dataset=love_train_dataset)
trainer_love.train()

trainer_love.save_model("emotion_fine_tuned/gpt2_love")
tokenizer_love.save_pretrained("emotion_fine_tuned/gpt2_love")

***** Running training *****
  Num examples = 1304
  Num Epochs = 10
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 6520
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: nastyatrvl (use `wandb login --relogin` to force relogin)


Step,Training Loss
500,4.488200
1000,3.842900
1500,3.486500
2000,3.042000
2500,2.581900
3000,2.189400
3500,1.961700
4000,1.690900
4500,1.475900
5000,1.313400


Saving model checkpoint to ./emotions/checkpoint-500
Configuration saved in ./emotions/checkpoint-500/config.json
Model weights saved in ./emotions/checkpoint-500/pytorch_model.bin
Saving model checkpoint to ./emotions/checkpoint-1000
Configuration saved in ./emotions/checkpoint-1000/config.json
Model weights saved in ./emotions/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to ./emotions/checkpoint-1500
Configuration saved in ./emotions/checkpoint-1500/config.json
Model weights saved in ./emotions/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to ./emotions/checkpoint-2000
Configuration saved in ./emotions/checkpoint-2000/config.json
Model weights saved in ./emotions/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to ./emotions/checkpoint-2500
Configuration saved in ./emotions/checkpoint-2500/config.json
Model weights saved in ./emotions/checkpoint-2500/pytorch_model.bin
Saving model checkpoint to ./emotions/checkpoint-3000
Configuration saved in ./emot

In [15]:
tokenizer_love = AutoTokenizer.from_pretrained("emotion_fine_tuned/gpt2_love")
love_model = AutoModelWithLMHead.from_pretrained("emotion_fine_tuned/gpt2_love")

In [16]:
gpt2_finetune_love = pipeline('text-generation', model=love_model, tokenizer=tokenizer_love)

## Fear

In [ ]:
tokenizer_fear = AutoTokenizer.from_pretrained('gpt2') 
model_fear = AutoModelWithLMHead.from_pretrained("gpt2")
model_fear.config.pad_token_id = model_fear.config.eos_token_id
tokenizer_fear.pad_token = tokenizer_fear.eos_token

data_collator_fear = DataCollatorForLanguageModeling(
        tokenizer=tokenizer_fear, mlm=False,
    )
trainer_fear = Trainer(
    model=model_fear, 
    args=training_args, 
    data_collator=data_collator_fear,
    train_dataset=fear_train_dataset)
trainer_fear.train()

trainer_fear.save_model("emotion_fine_tuned/gpt2_fear")
tokenizer_fear.save_pretrained("emotion_fine_tuned/gpt2_fear")

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/gpt2/resolve/main/config.json from cache at /home/avsorokina/.cache/huggingface/transformers/fc674cd6907b4c9e933cb42d67662436b89fa9540a1f40d7c919d0109289ad01.7d2e0efa5ca20cef4fb199382111e9d3ad96fd77b849e1d4bed13a66e1336f51
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_t

Step,Training Loss
500,4.433400
1000,4.102500
1500,3.593400
2000,3.491600
2500,3.008600
3000,2.987500
3500,2.492800
4000,2.420600
4500,2.092500
5000,2.026500


Saving model checkpoint to ./emotions/checkpoint-500
Configuration saved in ./emotions/checkpoint-500/config.json
Model weights saved in ./emotions/checkpoint-500/pytorch_model.bin
Saving model checkpoint to ./emotions/checkpoint-1000
Configuration saved in ./emotions/checkpoint-1000/config.json
Model weights saved in ./emotions/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to ./emotions/checkpoint-1500
Configuration saved in ./emotions/checkpoint-1500/config.json
Model weights saved in ./emotions/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to ./emotions/checkpoint-2000
Configuration saved in ./emotions/checkpoint-2000/config.json
Model weights saved in ./emotions/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to ./emotions/checkpoint-2500
Configuration saved in ./emotions/checkpoint-2500/config.json
Model weights saved in ./emotions/checkpoint-2500/pytorch_model.bin
Saving model checkpoint to ./emotions/checkpoint-3000
Configuration saved in ./emot

In [17]:
tokenizer_fear = AutoTokenizer.from_pretrained("emotion_fine_tuned/gpt2_fear")
fear_model = AutoModelWithLMHead.from_pretrained("emotion_fine_tuned/gpt2_fear")

In [18]:
gpt2_finetune_fear = pipeline('text-generation', model=fear_model, tokenizer=tokenizer_fear)

## Surprise

In [83]:
tokenizer_surprise = AutoTokenizer.from_pretrained('gpt2') 
model_surprise = AutoModelWithLMHead.from_pretrained("gpt2")
model_surprise.config.pad_token_id = model_surprise.config.eos_token_id
tokenizer_surprise.pad_token = tokenizer_surprise.eos_token

data_collator_surprise = DataCollatorForLanguageModeling(
        tokenizer=tokenizer_surprise, mlm=False,
    )
trainer_surprise = Trainer(
    model=model_surprise, 
    args=training_args, 
    data_collator=data_collator_surprise,
    train_dataset=surprise_train_dataset)
trainer_surprise.train()

trainer_surprise.save_model("emotion_fine_tuned/gpt2_surprise")
tokenizer_surprise.save_pretrained("emotion_fine_tuned/gpt2_surprise")

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/gpt2/resolve/main/config.json from cache at /home/avsorokina/.cache/huggingface/transformers/fc674cd6907b4c9e933cb42d67662436b89fa9540a1f40d7c919d0109289ad01.7d2e0efa5ca20cef4fb199382111e9d3ad96fd77b849e1d4bed13a66e1336f51
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_t

Step,Training Loss
500,4.313800
1000,3.300000
1500,2.396400
2000,1.705000
2500,1.222900


Saving model checkpoint to ./emotions/checkpoint-500
Configuration saved in ./emotions/checkpoint-500/config.json
Model weights saved in ./emotions/checkpoint-500/pytorch_model.bin
Saving model checkpoint to ./emotions/checkpoint-1000
Configuration saved in ./emotions/checkpoint-1000/config.json
Model weights saved in ./emotions/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to ./emotions/checkpoint-1500
Configuration saved in ./emotions/checkpoint-1500/config.json
Model weights saved in ./emotions/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to ./emotions/checkpoint-2000
Configuration saved in ./emotions/checkpoint-2000/config.json
Model weights saved in ./emotions/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to ./emotions/checkpoint-2500
Configuration saved in ./emotions/checkpoint-2500/config.json
Model weights saved in ./emotions/checkpoint-2500/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)

In [19]:
tokenizer_surprise = AutoTokenizer.from_pretrained("emotion_fine_tuned/gpt2_surprise")
surprise_model = AutoModelWithLMHead.from_pretrained("emotion_fine_tuned/gpt2_surprise")

In [20]:
gpt2_finetune_surprise = pipeline('text-generation', model=surprise_model, tokenizer=tokenizer_surprise)

## Anger

In [22]:
tokenizer_anger = AutoTokenizer.from_pretrained('gpt2') 
model_anger = AutoModelWithLMHead.from_pretrained("gpt2")
model_anger.config.pad_token_id = model_anger.config.eos_token_id
tokenizer_anger.pad_token = tokenizer_anger.eos_token

data_collator_anger = DataCollatorForLanguageModeling(
        tokenizer=tokenizer_anger, mlm=False,
    )
trainer_anger = Trainer(
    model=model_anger, 
    args=training_args, 
    data_collator=data_collator_anger,
    train_dataset=anger_train_dataset)
trainer_anger.train()

trainer_anger.save_model("emotion_fine_tuned/gpt2_anger")
tokenizer_anger.save_pretrained("emotion_fine_tuned/gpt2_anger")

In [23]:
tokenizer_anger = AutoTokenizer.from_pretrained("emotion_fine_tuned/gpt2_anger")
anger_model = AutoModelWithLMHead.from_pretrained("emotion_fine_tuned/gpt2_anger")

In [24]:
gpt2_finetune_anger = pipeline('text-generation', model=anger_model, tokenizer=tokenizer_anger)

# Experiments

In [25]:
test_prompts = ["I am ",
             "I think that",
             "I like",
             "I don't like",
             "I want",
             "My dream is"]

In [26]:
classifier = pipeline("text-classification",model='bhadresh-savani/distilbert-base-uncased-emotion', return_all_scores=True)

In [27]:
classifier2 = pipeline("text-classification",model='arpanghoshal/EmoRoBERTa', return_all_scores=True)

404 Client Error: Not Found for url: https://huggingface.co/arpanghoshal/EmoRoBERTa/resolve/main/pytorch_model.bin
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at arpanghoshal/EmoRoBERTa.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [34]:
for prompt in test_prompts:
    res = gpt2_finetune_anger(text_inputs=prompt, 
                      top_k = 50,
                      top_p = 0.95,
                      no_repeat_ngram_size = 3,  
                      repetition_penalty=1.2,
                      max_length = 50,
                      temperature = 0.7)
    prediction1 = classifier(res[0]['generated_text'])
    prediction2 = classifier2(res[0]['generated_text'])
    prediction1 = sorted(prediction1[0], key=lambda d: d['score'], reverse = True) 
    prediction2 = sorted(prediction2[0], key=lambda d: d['score'], reverse = True) 
    print(res[0]['generated_text'], 2*"\n", prediction1[0], 2*"\n", prediction2[0])
    print(100*'-')

I am ive been feeling pretty grumpy lately with school work so i figured it would be a good time to get my hackles up about that too ehb? hmmmmm what is school do britain if youre into film study 

 {'label': 'anger', 'score': 0.9981691837310791} 

 {'label': 'realization', 'score': 0.9645422101020813}
----------------------------------------------------------------------------------------------------
I think that feeling dissatisfied with my life transformed into this state where i am miserable and depressed about everything most of the time even though i have been through a lot without much change in me or at least not much satisfaction yet again because all i ever want 

 {'label': 'anger', 'score': 0.9977060556411743} 

 {'label': 'disappointment', 'score': 0.9583703279495239}
----------------------------------------------------------------------------------------------------
I like to buck the system and climb on my soapbox when i feel wronged or see others wronged but lately i fe

In [29]:
for prompt in test_prompts:
    res = gpt2_finetune_joy(text_inputs=prompt, 
                      top_k = 50,
                      top_p = 0.95,
                      no_repeat_ngram_size = 3,  
                      repetition_penalty=1.2,
                      max_length = 30,
                      temperature = 0.7)
    prediction1 = classifier(res[0]['generated_text'])
    prediction2 = classifier2(res[0]['generated_text'])
    prediction1 = sorted(prediction1[0], key=lambda d: d['score'], reverse = True) 
    prediction2 = sorted(prediction2[0], key=lambda d: d['score'], reverse = True) 
    print(res[0]['generated_text'], 2*"\n", prediction1[0], 2*"\n", prediction2[0])
    print(100*'-')

I am ive always felt that i couldnt be more appreciative of people who made me feel unhappy than the ones we all have to think twice 

 {'label': 'sadness', 'score': 0.9982587695121765} 

 {'label': 'disappointment', 'score': 0.667558491230011}
----------------------------------------------------------------------------------------------------
I think that s how we should feel when something big has happened in our life and it feels like things are getting resolved a bit too quickly for this 

 {'label': 'joy', 'score': 0.9986312985420227} 

 {'label': 'approval', 'score': 0.6539403200149536}
----------------------------------------------------------------------------------------------------
I like to listen and feel entertained by the singers since i know they are so popular but as a group effort it feels like a chore not being present 

 {'label': 'joy', 'score': 0.9989572763442993} 

 {'label': 'joy', 'score': 0.9555159211158752}
-----------------------------------------------------

In [30]:
for prompt in test_prompts:
    res = gpt2_finetune_love(text_inputs=prompt, 
                      top_k = 50,
                      top_p = 0.95,
                      no_repeat_ngram_size = 3,  
                      repetition_penalty=1.2,
                      max_length = 50,
                      temperature = 0.7)
    prediction1 = classifier(res[0]['generated_text'])
    prediction2 = classifier2(res[0]['generated_text'])
    prediction1 = sorted(prediction1[0], key=lambda d: d['score'], reverse = True) 
    prediction2 = sorted(prediction2[0], key=lambda d: d['score'], reverse = True) 
    print(res[0]['generated_text'], 2*"\n", prediction1[0], 2*"\n", prediction2[0])
    print(100*'-')

I am ive been feeling a bit nostalgic lately because i m no longer trying to rekindle some sort of nostalgia craze in my life but rather rather ively rekindling the desire for something more than just another piece back to the family 

 {'label': 'love', 'score': 0.9944379329681396} 

 {'label': 'realization', 'score': 0.8780173063278198}
----------------------------------------------------------------------------------------------------
I think that the woman in front of me should have said no way and if she feels like telling a stranger this is what kind people feel towards them it will be treated with dignity and respect as though i gave myself permission to enter into some sort contract 

 {'label': 'joy', 'score': 0.5518168807029724} 

 {'label': 'realization', 'score': 0.595809817314148}
----------------------------------------------------------------------------------------------------
I like to do it makes me feel very out of control but sometimes i just have to get in and take

In [31]:
# for prompt in test_prompts:
#     print(gpt2_finetune_joy2(text_inputs=prompt, 
#                       top_k = 50,
#                       top_p = 0.95,
#                       no_repeat_ngram_size = 3,  
#                       repetition_penalty=1.2,
#                       max_length = 50,
#                       temperature = 0.7)
#          )

In [32]:
for prompt in test_prompts:
    res = gpt2_finetune_surprise(text_inputs=prompt, 
                      top_k = 50,
                      top_p = 0.95,
                      no_repeat_ngram_size = 3,  
                      repetition_penalty=1.2,
                      max_length = 50,
                      temperature = 0.7)
    prediction1 = classifier(res[0]['generated_text'])
    prediction2 = classifier2(res[0]['generated_text'])
    prediction1 = sorted(prediction1[0], key=lambda d: d['score'], reverse = True) 
    prediction2 = sorted(prediction2[0], key=lambda d: d['score'], reverse = True) 
    print(res[0]['generated_text'], 2*"\n", prediction1[0], 2*"\n", prediction2[0])
    print(100*'-')

I am ive always felt amazed by how they can do something once i started to follow a set of food rules and adapt quickly without any trouble or stressor while balancing out my calorie needs efficiently with what is left over from the day we were here 

 {'label': 'surprise', 'score': 0.9907650947570801} 

 {'label': 'surprise', 'score': 0.9749290347099304}
----------------------------------------------------------------------------------------------------
I think that after all these years it still feels strange to realize how much i inspired people through my work and the friendships I made throughout this amazing life. ive always wanted to be a part of something bigger than myself but now im just waking up 

 {'label': 'surprise', 'score': 0.8423177599906921} 

 {'label': 'realization', 'score': 0.9772616624832153}
----------------------------------------------------------------------------------------------------
I like to do things that leave others feeling amazed and delighted whil

In [33]:
for prompt in test_prompts:
    res = gpt2_finetune_fear(text_inputs=prompt, 
                      top_k = 50,
                      top_p = 0.95,
                      no_repeat_ngram_size = 3,  
                      repetition_penalty=1.2,
                      max_length = 50,
                      temperature = 0.7)
    prediction1 = classifier(res[0]['generated_text'])
    prediction2 = classifier2(res[0]['generated_text'])
    prediction1 = sorted(prediction1[0], key=lambda d: d['score'], reverse = True) 
    prediction2 = sorted(prediction2[0], key=lambda d: d['score'], reverse = True) 
    print(res[0]['generated_text'], 2*"\n", prediction1[0], 2*"\n", prediction2[0])
    print(100*'-')

I am ive had so many bad experiences that i feel intimidated to write about them here because ill be too busy to bother with the rest of this post at all times and want you guys to know how ive gone through the tough moments before coming 

 {'label': 'fear', 'score': 0.997031569480896} 

 {'label': 'fear', 'score': 0.9451883435249329}
----------------------------------------------------------------------------------------------------
I think that the main benefit of using a clinique product is having an easy way to add more products without feeling pressured to because there are so many others out there who share my vision and feel like they need it for their work life balance or if 

 {'label': 'fear', 'score': 0.9976527094841003} 

 {'label': 'approval', 'score': 0.9627368450164795}
----------------------------------------------------------------------------------------------------
I like to add some cute photos from today as well so i can keep doing what i am doing and not feel shy